In [7]:
import pandas as pd
import geopandas as gpd
import urllib
import json
import pycountry

In [8]:
# JSON time-series of coronavirus cases (confirmed, deaths and recovered) per country - updated daily on github
with urllib.request.urlopen("https://pomber.github.io/covid19/timeseries.json") as url:
    data = json.loads(url.read().decode())

In [23]:
# creates a data frame for the option selected choice between confirmed cases deaths or recoveries
def corona_stats(option):   
    options = ['confirmed', 'deaths', 'recovered']
    if option not in options: option = option[0]
        
    columns = ['entity', 'code', 'dates', option]
    gdata = []
    for c in data.keys():
        for d in range(len(data[c])):
            country = pycountry.countries.get(name =c)
            if(country== None): country = pycountry.countries.get(common_name =c)
            if(country != None):
                       code = country.alpha_3
            if(c == 'US'): code = "USA"
            if(c == 'Russia'): code = "RUS"
            if(c == 'Vietnam'): code = "VNM"
            if(c == 'Taiwan*'): code = "TWN"
#             print(data[c][d])
            gdata.append([c, code, data[c][d]['date'], data[c][d][option]])
    df = pd.DataFrame(gdata,columns=columns)
    return df

df = corona_stats('confirmed')
print(df)

            entity code      dates  confirmed
0      Afghanistan  AFG  2020-1-22          0
1      Afghanistan  AFG  2020-1-23          0
2      Afghanistan  AFG  2020-1-24          0
3      Afghanistan  AFG  2020-1-25          0
4      Afghanistan  AFG  2020-1-26          0
...            ...  ...        ...        ...
16645        Yemen  YEM  2020-4-16          1
16646        Yemen  YEM  2020-4-17          1
16647        Yemen  YEM  2020-4-18          1
16648        Yemen  YEM  2020-4-19          1
16649        Yemen  YEM  2020-4-20          1

[16650 rows x 4 columns]


In [24]:
# loads countries for the map into a df and gets rid of antartica
shapefile = '../data/countries_110m/ne_110m_admin_0_countries.shp'
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
gdf.columns = ['country', 'country_code', 'geometry']
print(gdf.head())
gdf = gdf.drop(gdf.index[159])

                       country country_code  \
0                         Fiji          FJI   
1  United Republic of Tanzania          TZA   
2               Western Sahara          SAH   
3                       Canada          CAN   
4     United States of America          USA   

                                            geometry  
0  MULTIPOLYGON (((180.00000 -16.06713, 180.00000...  
1  POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...  
2  POLYGON ((-8.66559 27.65643, -8.66512 27.58948...  
3  MULTIPOLYGON (((-122.84000 49.00000, -122.9742...  
4  MULTIPOLYGON (((-122.84000 49.00000, -120.0000...  


In [12]:
def json_data(date):
    df = df[df['dates'] == date]
    merged = gdf.merge(df, left_on = 'country_code', right_on = 'code', how = 'left')
    merged.fillna('No data', inplace = True)
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    return json_data


        country country_code  \
159  Antarctica          ATA   

                                              geometry  
159  MULTIPOLYGON (((-48.66062 -78.04702, -48.15140...  
